In [2]:
import requests
import pybel
import pybel_tools
from pybel_tools import summary
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import re
import yaml

import tqdm
from tqdm import tqdm # not sure why you need both

import os
import json
import numpy as np
import collections
import matplotlib.pyplot as plt
import seaborn as sns

#from indra.processors import bel
from indra.sources import bel
from indra.util import batch_iter
from indra.sources import indra_db_rest

In [3]:
pybel_pd=pd.read_csv('/home/lani_lichtenstein/covid19_frauenhofer_annotations_entities.csv',index_col=0)

In [4]:
#First create columns in pybel_pd for source and target ids (without extra information )

# extract name value from the source

src_ids=[]
for i in range(len(pybel_pd)):
    src_tmp=pybel_pd.iloc[i]['source'] # so we are told that tetrandine is the source... - but where is it?
    src_tmp=src_tmp.replace( '\'namespace\'' , '\"namespace\"')
    src_tmp=src_tmp.replace( '\'name\'' , '\"name\"')
    src_tmp=src_tmp.replace( '\'identifier\'' , '\"identifier\"')
    src_tmp=src_tmp.replace( '{\'' , '{\"')
    src_tmp=src_tmp.replace( '\'}' , '\"}')
    src_tmp=src_tmp.replace( '\':' , '\":')
    src_tmp=src_tmp.replace( ': \'' , ': \"')
    src_tmp=src_tmp.replace( '\', ' , '\", ')    
    src_tmp=src_tmp.replace( ', \'', ', \"')

    #src_tmp=src_tmp.replace( '\'' , '\"')
    
    src_dict=json.loads(src_tmp)

    src_ids_tmp_list=[]
    for key,val in src_dict.items():
        src_name=val['name']
        src_id=val['identifier']
        src_id_name=src_name + "_Id:" + src_id
        src_ids_tmp_list.append(src_id_name)
        
    src_ids.append(src_ids_tmp_list)

In [5]:
# extract name value from the target

trgt_ids=[]
for i in range(len(pybel_pd)):
    trgt_tmp=pybel_pd.iloc[i]['target'] # so we are told that tetrandine is the source... - but where is it?

    trgt_tmp=trgt_tmp.replace( '\'namespace\'' , '\"namespace\"')
    trgt_tmp=trgt_tmp.replace( '\'name\'' , '\"name\"')
    trgt_tmp=trgt_tmp.replace( '\'identifier\'' , '\"identifier\"')
    trgt_tmp=trgt_tmp.replace( '{\'' , '{\"')
    trgt_tmp=trgt_tmp.replace( '\'}' , '\"}')
    trgt_tmp=trgt_tmp.replace( '\':' , '\":')
    trgt_tmp=trgt_tmp.replace( ': \'' , ': \"')
    trgt_tmp=trgt_tmp.replace( '\', ' , '\", ')    
    trgt_tmp=trgt_tmp.replace( ', \'', ', \"')

    #src_tmp=src_tmp.replace( '\'' , '\"')
    trgt_dict=json.loads(trgt_tmp)

    trgt_ids_tmp_list=[]
    for key,val in trgt_dict.items():
        trgt_name=val['name']
        trgt_id=val['identifier']
        trgt_id_name= trgt_name + "_Id:" + trgt_id
        trgt_ids_tmp_list.append(trgt_id_name)
            
    trgt_ids.append(trgt_ids_tmp_list)
    
 
# Note - problem entry, identifier same as name  - use pybel to resolve
#{"3.4.22.69": {"namespace": "eccode", "name": "3.4.22.69", "identifier": "3.4.22.69"}}
# note there are few extra src ids because -some in pd_dataframe appear twice 


In [6]:
# append  new columns to pybel_pd
pybel_pd['src_identifier_list']=pd.Series(src_ids,index=pybel_pd.index)
pybel_pd['trgt_identifier_list']=pd.Series(trgt_ids,index=pybel_pd.index)

In [7]:
# Create some dictionaries

# identifier_synonym_dict - for each unique identifer - get a list of tuples, 
#   where each tuple contains (all matched entities, sentence text, tokenized sentence text) 

from collections import defaultdict, Counter
identifier_synonym_dict = defaultdict(list)
identifier_counter=Counter()

for i,trgt_list_tmp  in enumerate(trgt_ids): # for each element in pybel target list
        entities_tmp=pybel_pd.iloc[i]['matched_entities'] 
        text_tmp=pybel_pd.iloc[i]['text'] 
        tok_text_tmp=pybel_pd.iloc[i]['tokenized_texts'] 

        
        for j,trgt_tmp in enumerate(trgt_list_tmp): # for each element in this list of 1 or more targets, for a given text
            identifier_synonym_dict[trgt_tmp].append((entities_tmp,text_tmp,tok_text_tmp)) # may be adding text to more than one target, 
            identifier_counter.update({trgt_tmp:1})
            
# no need for 2 for loops - combine this above and go through each row of pybel_pd once.. 
for i, src_list_tmp  in enumerate(src_ids): # for each element in pybel target list
        entities_tmp=pybel_pd.iloc[i]['matched_entities'] 
        text_tmp=pybel_pd.iloc[i]['text'] 
        tok_text_tmp=pybel_pd.iloc[i]['tokenized_texts'] 

        
        for j,src_tmp in enumerate(src_list_tmp): # for each element in this list of 1 or more srces, for a given text
            identifier_synonym_dict[src_tmp].append((entities_tmp,text_tmp,tok_text_tmp)) # may be adding text to more than one target, 
            identifier_counter.update({src_tmp:1})

In [9]:
list(identifier_synonym_dict.items())[0]

('Tpcn2_Id:2385297',
 [('[{"{\'token\': \'TPC2 ,\', \'string\': \'TPC2 ,\', \'start_token\': 44, \'end_token\': 46, \'umls_id\': \'C1421947\', \'canonical_name\': \'MCOLN1 gene\', \'start_char\': 240, \'end_char\': 246}", "{\'token\': \'TPC2\', \'string\': \'TPC2\', \'start_token\': 2, \'end_token\': 3, \'umls_id\': \'C1427150\', \'canonical_name\': \'TPCN2 gene\', \'start_char\': 15, \'end_char\': 19}", "{\'token\': \'tetrandrine\', \'string\': \'tetrandrine\', \'start_token\': 5, \'end_token\': 6, \'umls_id\': \'C0076316\', \'canonical_name\': \'tetrandrine\', \'start_char\': 32, \'end_char\': 43}"}]',
   'While blocking TPC2 activity by tetrandrine, an inhibitor for TPC237, decreased entry of SARS-CoV-2 S pseudovirions (Fig. 3f), treatment of cells with 130, a TRPML1 inhibitor, had no effect (Supplementary Fig. 1), indicating that TPC2, not TRPML1, is important for SARS-CoV-2 entry.',
   "[['While', 'blocking', 'TPC2', 'activity', 'by', 'tetrandrine', ',', 'an', 'inhibitor', 'for', 

In [11]:
for letter, count in identifier_counter.most_common(10):
    print('%s: %d' % (letter, count)) ## ok
    
# this is the number of sentence text with each source identifier. 
#Note counted more than once if identifier appears in target as well. 

COVID-19_Id:0080600: 2796
Severe acute respiratory syndrome-related coronavirus_Id:694009: 213
ACE2_Id:13557: 158
chloroquine_Id:3638: 121
AP3A_CVHSA_Id:P59632: 118
Mortality_Id:D009026: 98
adult respiratory distress syndrome_Id:11394: 81
IL6_Id:6018: 75
TNNT2_Id:11949: 70
R1AB_CVHSA_Id:P0C6X7: 67


In [25]:
# For each grounded identifier key in the dictionary, get all the associated sentence data, 
# where each assoc sentnece data consists of
# - a sentence, and the entities retrieved for that sentence
# ---  For each matched entity for the associated sentend data, what is the text at that location? 
# ---- That text is added as a 'potential' synonym for this identifier though likely a false positive

counter=0
i=0
identifier_theasaurus=defaultdict(list)
for key, list_of_tuples in identifier_synonym_dict.items(): # key is grounded identifier
    i+=1
    for identifier_tuple in list_of_tuples: # for each sentence (row in pybel_pd) that matches this identifier
        #ents,txt,tok_txt=identifier_synonym_dict['2385297'][0] # get from 
        ents,txt,tok_txt=identifier_tuple

    if not ents=='[set()]': # if entity list is not empty 
        ents1=ents.replace('\'','\"')

        ents1=ents1.replace('{\"{','{')
        ents1=ents1.replace('\"{\"','{\"')

        ents1=ents1.replace('}\"}','}')
        ents1=ents1.replace('}\",','},')
        
        try:
            ents_list=json.loads(ents1) # a list of dicts
            for ents_dict_tmp in ents_list: # for each entity in the list, # dont know if this is for src or target
                start=ents_dict_tmp['start_token']
                end=ents_dict_tmp['end_token']
                synonym_tmp=tok_txt.split()[start:(end)]
                identifier_theasaurus[key].append(synonym_tmp)  # so adding the entity original text as a synonym for 
            
        except ValueError:  
            counter+=1
            pass


In [26]:
for key, val in identifier_theasaurus.items():
    print(key)
    print(val)
    print("\n")

Tpcn2_Id:2385297
[["'TPC2',", "',',"], ["'TPC2',"], ["'tetrandrine',"]]


3.4.22.69_Id:3.4.22.69
[["'lopinavir/ritonavir',", "'in',", "'the',", "'spatial',"]]


HAS2_Id:4819
[["'HA-synthase-2',", "'(',"]]


inflammatory response_Id:0006954
[["'pathogenesis',", "'of',", "'inflammatory',"]]


Death_Id:D003643
[["'Acute',", "'Respiratory',", "'Distress',", "'Syndrome',", "'(',"], ["'Respiratory',", "'Distress',", "'Syndrome',"], ["'Acute',", "'Respiratory',", "'Distress',", "'Syndrome',"]]


Nausea_Id:D009325
[["[['ACE2',"], ["'nausea',"]]


Vomiting_Id:D014839
[["[['ACE2',"]]


Cytokines_Id:D016207
[["'regulatory',", "'T',"], ["'of',", "'regulatory',", "'T',"], ["'cytokines',", "'in',"], ["'cytokines',", "'in',", "'COVID-19',"]]


atherosclerosis_Id:1936
[["'atherosclerosis',"]]


Endothelial Cells_Id:D042783
[["'angiopoietin',", "'ligands',", "'(ANGPT1/2)–Tie',"]]


RNA, Viral_Id:D012367
[["'viral',"], ["'VeroE6/TMPRSS2',", "'cell',"], ["'VeroE6/TMPRSS2',"]]


Low-grade fever_Id:0011134

Decreased basophil count_Id:0031808
[["'basophil',"], ["'ACE2',"]]


Pneumonia, Ventilator-Associated_Id:D053717
[["'ventilator-associated',"]]


Social Isolation_Id:D012934
[["'isolation',"]]


Convalescence_Id:D003289
[["'inflammation',"]]


